In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install langchain-google-genai==2.0.7

In [ ]:
!pip install langchain==0.3.11

In [ ]:
import os
import yaml

with open('gemini_key.yml', 'r') as file:
  api_creds = yaml.safe_load(file)

os.environ['GOOGLE_API_KEY'] = api_creds['gemini_key']

In [ ]:
api_creds.keys()

dict_keys(['gemini_key'])

### Load necessary libraries and Google Gemini LLM

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
gemini_model = ChatGoogleGenerativeAI(model = "gemini-pro", convert_system_message_to_human = True)

### Create a basic Chain

In [ ]:
PROMPT = "tell me a joke about {topic}"
prompt = ChatPromptTemplate.from_template(PROMPT)

chain = (
         prompt
         |
         gemini_model
)
response = chain.invoke({"topic": "AI"})
print(response.content)

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Why did the AI cross the road?

To get to the other algorithm.


In [ ]:
# can be used on multiple prompts also
topics = [{"topic":"Gen_AI"}, {"topic":"Statistics"}]

responses = chain.map().invoke(topics)

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [ ]:
for response in responses:
  print(response.content)
  print("-"*100)
  print("\n")

Why did Gen_AI cross the road?

To get to the other prompt!
----------------------------------------------------------------------------------------------------


Why did the statistician get lost in the forest?

Because he took the average turn.
----------------------------------------------------------------------------------------------------




### Basic Chains are ad-hoc, No Conversation history

In [ ]:
prompt = ChatPromptTemplate.from_template("{query}")

basic_chain =(
              prompt
              |
              gemini_model
)

response = basic_chain.invoke({"query": "what are the first 4 planets in the solar system?"})
print(response.content)

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


1. Mercury
2. Venus
3. Earth
4. Mars


In [ ]:
response = basic_chain.invoke({"query": "And the last three?"})
print(response.content)

1. **Dark Side of the Moon** by Pink Floyd (1973)
2. **Thriller** by Michael Jackson (1982)
3. **Back in Black** by AC/DC (1980)


#### Here, we can see that the second response generated has no contextual correlation with the previous response.

### Add Memory to build a conversation chain

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [ ]:
# Create Prompt Template
prompt = ChatPromptTemplate.from_messages(
     [
         ("system", "Act as a helpful AI Assistant"),
         MessagesPlaceholder(variable_name = "history"),
         ("human", "{input}")
     ]
)

# k=3 stores last 3 conversations between you and gemini
memory = ConversationBufferWindowMemory(k=3, return_messages = True)

In [ ]:
memory.load_memory_variables({}) # Shows the conversation history

{'history': []}

In [ ]:
from operator import itemgetter

# Create the conversation chain
chain = (
         RunnablePassthrough.assign(
             history = RunnableLambda(memory.load_memory_variables)
             |
             itemgetter("history")
         )
         |
         prompt
         |
         gemini_model
)

In [ ]:
user_input = {"input": "what are the first 4 planets in the solar system"}
response = chain.invoke(user_input)
response.content

'1. Mercury\n2. Venus\n3. Earth\n4. Mars'

In [ ]:
memory.load_memory_variables({})

{'history': []}

In [ ]:
# remember to save your conversation to the memory
memory.save_context(user_input, {"output": response.content})
memory.load_memory_variables({}) # remembers the conversation history

{'history': [HumanMessage(content='what are the first 4 planets in the solar system', additional_kwargs={}, response_metadata={}),
  AIMessage(content='1. Mercury\n2. Venus\n3. Earth\n4. Mars', additional_kwargs={}, response_metadata={})]}

In [ ]:
user_input = {"input": "And the last 4?"}
response = chain.invoke(user_input)
print(response.content)

The last 4 planets in the solar system are:

1. Jupiter
2. Saturn
3. Uranus
4. Neptune


In [ ]:
memory.save_context(user_input, {"output":response.content})
memory.load_memory_variables({})

{'history': [HumanMessage(content='what are the first 4 planets in the solar system', additional_kwargs={}, response_metadata={}),
  AIMessage(content='1. Mercury\n2. Venus\n3. Earth\n4. Mars', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='And the last 4?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The last 4 planets in the solar system are:\n\n1. Jupiter\n2. Saturn\n3. Uranus\n4. Neptune', additional_kwargs={}, response_metadata={})]}

In [ ]:
user_input = {"input": "which is the largest among all?"}
response = chain.invoke(user_input)
print(response.content)

Jupiter is the largest planet in the solar system. It is a gas giant with a mass that is more than twice the mass of all the other planets in the solar system combined. Jupiter is also the fastest rotating planet in the solar system, with a day that lasts only about 10 hours.

Here is a table of the sizes of the planets in the solar system, from largest to smallest:

| Planet | Diameter (km) |
|---|---|
| Jupiter | 142,984 |
| Saturn | 116,464 |
| Uranus | 50,724 |
| Neptune | 49,244 |
| Earth | 12,742 |
| Venus | 12,104 |
| Mars | 6,779 |
| Mercury | 4,879 |


In [ ]:
memory.save_context(user_input, {"output": response.content})
memory.load_memory_variables({})

{'history': [HumanMessage(content='what are the first 4 planets in the solar system', additional_kwargs={}, response_metadata={}),
  AIMessage(content='1. Mercury\n2. Venus\n3. Earth\n4. Mars', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='And the last 4?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The last 4 planets in the solar system are:\n\n1. Jupiter\n2. Saturn\n3. Uranus\n4. Neptune', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='which is the largest among all?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Jupiter is the largest planet in the solar system. It is a gas giant with a mass that is more than twice the mass of all the other planets in the solar system combined. Jupiter is also the fastest rotating planet in the solar system, with a day that lasts only about 10 hours.\n\nHere is a table of the sizes of the planets in the solar system, from largest to smallest:\n\n| Planet | Diame

In [ ]:
user_input = {"input": "And the smallest?"}
response = chain.invoke(user_input)
print(response.content)

Mercury is the smallest planet in the solar system. It is a terrestrial planet with a diameter of only 4,879 kilometers (3,031 miles). Mercury is also the closest planet to the Sun, and it has a very thin atmosphere.

Here is a table of the sizes of the planets in the solar system, from smallest to largest:

| Planet | Diameter (km) |
|---|---|
| Mercury | 4,879 |
| Mars | 6,779 |
| Venus | 12,104 |
| Earth | 12,742 |
| Neptune | 49,244 |
| Uranus | 50,724 |
| Saturn | 116,464 |
| Jupiter | 142,984 |


In [ ]:
user_input = {"input": "which is the fastest?"}
response = chain.invoke(user_input)
print(response.content)

Jupiter is the fastest rotating planet in the solar system. It takes Jupiter only about 10 hours to complete one rotation, while it takes Earth 24 hours to complete one rotation. This means that a day on Jupiter is only about 10 hours long.

Here is a table of the rotation periods of the planets in the solar system, from fastest to slowest:

| Planet | Rotation period (hours) |
|---|---|
| Jupiter | 10 |
| Saturn | 10.7 |
| Uranus | 17.2 |
| Neptune | 16.1 |
| Earth | 24 |
| Venus | 243 |
| Mars | 24.6 |
| Mercury | 58.6 |

As you can see, Jupiter's rotation period is much shorter than the rotation periods of the other planets in the solar system. This is because Jupiter is a gas giant, and gas giants rotate much faster than rocky planets.


In [ ]:
user_input = {"input": "which is the slowest?"}
response = chain.invoke(user_input)
print(response.content)

Mercury is the slowest planet in the solar system. It takes Mercury 88 Earth days to orbit the Sun, which is the longest orbital period of any planet in the solar system. Mercury's slow orbit is due to its proximity to the Sun. The Sun's gravity exerts a strong pull on Mercury, which slows down its orbit.

Here is a table of the orbital periods of the planets in the solar system, from slowest to fastest:

| Planet | Orbital period (Earth days) |
|---|---|
| Mercury | 88 |
| Venus | 225 |
| Earth | 365 |
| Mars | 687 |
| Jupiter | 4,333 |
| Saturn | 10,759 |
| Uranus | 30,685 |
| Neptune | 60,190 |
